#### Hypothesis testing amounts difference of fraudulent transactions ####

Checking if sender balance features differ in fraudulent transactions in comparison with non-fraudulent
- H0: distribution of sender balance fetures in Fraud and Non-Fraud transactions are same
- H1:  distribution of sender balance fetures in Fraud and Non-Fraud transactions significantly differ

In [3]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()

conn_params = {
    "host": os.getenv("POSTGRES_HOST"),
    "port": os.getenv("POSTGRES_PORT"),
    "dbname": os.getenv("POSTGRES_DB"),
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD"),
}

conn = psycopg2.connect(**conn_params)

#cur = conn.cursor()

query = """
    SELECT 
        transaction_id,
        amount,
        sender_balance_before,
        sender_balance_after,
        (sender_balance_before - sender_balance_after) sender_balance_delta,
        is_fraud
    FROM 
        analytics_staging.stg_transactions
     WHERE sender_balance_before is not null 
       AND sender_balance_after is not null
     
"""

#cur.execute(query)
#data = cur.fetchall()

df = pd.read_sql(query, conn)

df.describe()


/tmp/ipykernel_67505/3163806764.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,transaction_id,amount,sender_balance_before,sender_balance_after,sender_balance_delta
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,3.499441e+07,1.798619e+05,8.338831e+05,8.551137e+05,-2.123056e+04
std,1.836730e+06,6.038582e+05,2.888243e+06,2.924049e+06,1.466433e+05
min,3.181310e+07,0.000000e+00,0.000000e+00,0.000000e+00,-1.915268e+06
25%,3.340376e+07,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.499441e+07,7.487194e+04,1.420800e+04,0.000000e+00,0.000000e+00
75%,3.658507e+07,2.087215e+05,1.073152e+05,1.442584e+05,1.015044e+04
max,3.817572e+07,9.244552e+07,5.958504e+07,4.958504e+07,1.000000e+07


In [ ]:
# bool to int for is_fraud
df["is_fraud"] = df["is_fraud"].astype(int)

# split to fruad_positive (actual fraud) and fraud_negative (non-fraud) transactions
fruad_positive = df[df['is_fraud'] == 1]
fraud_negative = df[df['is_fraud'] == 0]

print("Rows:", len(df), "Fraud:", len(fruad_positive), "Non-fraud:", len(fraud_negative))

df.head()

Rows: 6362620 Fraud: 8213 Non-fraud: 6354407


,transaction_id,amount,sender_balance_before,sender_balance_after,sender_balance_delta,is_fraud
0,31813101,9839.64,170136.0,160296.36,9839.64,0
1,31813102,1864.28,21249.0,19384.72,1864.28,0
2,31813103,181.00,181.0,0.00,181.00,1
3,31813104,181.00,181.0,0.00,181.00,1
4,31813105,11668.14,41554.0,29885.86,11668.14,0


In [15]:
import numpy as np
from scipy import stats

# Cliff's delta p(X > Y) - p(X < Y)
# range [-1, 1] where values close to 0 are values with no stochastatic dominance
def cliffs_delta(x:np.ndarray, y:np.ndarray) -> float:
    gt = 0
    lt = 0

    for x_i in x:
        gt += np.sum(x_i > y)
        lt += np.sum(x_i < y)
    return (gt - lt) / (len(x) - len(y))


def check_feature_stats(feature:str):
    x = fruad_positive[feature].dropna().to_numpy()
    y = fraud_negative[feature].dropna(). to_numpy()

    out = {}

    # Mann-Whitney two-sided test
    u_stat, u_p = stats.mannwhitneyu(x, y, alternative="two-sided")
    out["mannwhitney_u"] = u_stat
    out["mannwhitney_p"] = u_p

    # Welch t-test two-sided independent
    t_stat, t_p = stats.ttest_ind(x, y, equal_var=False, nan_policy="omit")
    out["welch_t"] = t_stat
    out["welch_p"] = t_p

    return out


In [20]:
# running test
features = ["sender_balance_before", "sender_balance_after", "sender_balance_delta"]

stat_tests_results = pd.DataFrame(check_feature_stats(f) for f in features)

stat_tests_results.head()

,mannwhitney_u,mannwhitney_p,welch_t,welch_p
0,4.233796e+10,0.0,20.857020,3.495452e-94
1,1.546039e+10,0.0,-30.550072,2.691860e-194
2,5.038765e+10,0.0,55.992451,0.000000e+00


In [24]:
print(df.groupby("is_fraud")[["sender_balance_before","sender_balance_after","sender_balance_delta"]].median())

          sender_balance_before  sender_balance_after  sender_balance_delta
is_fraud                                                                   
0                      14069.00                   0.0                  0.00
1                     438983.45                   0.0             436317.49


- According to independent t-test result for sender_balance_delta between Fraud and Non-Fraud transactions - H0 is rejected. 
- Sender balance delta strongly differs - consistent with balance transition anomalies.